In [ ]:
import os,sys
import kfp
import json

### User Variables

In [ ]:
## Fill in this cell with the repo names that you have chosen.

code_repo = "mnist"
dataset_repo = "mnist"
model_repo = "mnist"

### Get/Set Environment Variables

In [ ]:
components_url = "/mnt/dkube/pipeline/components/"
dkube_training_op = kfp.components.load_component_from_file(components_url + "training/component.yaml")
dkube_serving_op = kfp.components.load_component_from_file(components_url + "serving/component.yaml")
token = os.getenv("DKUBE_USER_ACCESS_TOKEN")
project_id = os.environ.get("DKUBE_PROJECT_ID")
project_name = os.environ.get("DKUBE_PROJECT_NAME")
user_name = os.environ.get("DKUBE_USER_LOGIN_NAME")
training_execution_image = "ocdr/dkube-datascience-tf-cpu:v2.0.0-17"
client = kfp.Client(host=os.getenv("KF_PIPELINES_ENDPOINT"), existing_token=token, namespace=os.getenv("USERNAME"))
run_id = 0

### Setup Pipeline Definitions

In [ ]:
@kfp.dsl.pipeline(
    name='dkube-mnist-pl',
    description='sample mnist pipeline with dkube components'
)
def mnist_pipeline(program=code_repo, dataset=dataset_repo, model=model_repo, project_id='abc123'):

    train       = dkube_training_op(container='{"image":training_execution_image}',
                                    framework="tensorflow", version="2.0.0",
                                    tags=tags,
                                    program=str(program), run_script="python mnist/train.py",
                                    datasets=json.dumps([str(dataset)]), outputs=json.dumps([str(model)]),
                                    input_dataset_mounts='["/mnist"]',
                                    output_mounts='["/model"]',
                                    envs='[{"EPOCHS": "1"}]',
                                    auth_token=token)

    serving     = dkube_serving_op(model=train.outputs['artifact'], device='cpu', 
                                    serving_image='{"image":"ocdr/tensorflowserver:2.0.0"}',
                                    transformer_image='{"image":training_execution_image}',
                                    transformer_project=str(program),
                                    transformer_code='mnist/transformer.py', auth_token=token).after(train)

### Create Pipeline Run Name

In [ ]:
if project_id:
    tags = json.dumps([f"project:{project_id}"])
    run_name = f"[{project_name}] {user_name} mnist Run{run_id}"
    experiment = f"[{project_name}]"
else:
    tags = []
    run_name = f"{user_name} mnist Run{run_id}"
    experiment = "default"

client.create_run_from_pipeline_func(mnist_pipeline, run_name=run_name, experiment_name=experiment, arguments={"token":token, "tags":tags})
run_id += 1

### Submit Pipeline

In [ ]:
#generate & upload pipeline (Optional)
#import kfp.compiler as compiler
#compiler.Compiler().compile(mnist_pipeline, f"[{project_name}]-pipeline.zip")
#client.upload_pipeline(f"[{project_name}]-pipeline.zip")